In [28]:
import sys
sys.path.append("/home/jacopo.gasparetto/Workspaces/keras-yolo2")
import numpy as np
import os, cv2

from keras.models import Model
from keras.layers import Input, Activation, Concatenate
from keras.layers import Flatten, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers.merge import concatenate


from keras.preprocessing.image import ImageDataGenerator, load_img

from keras.optimizers import Adam
from keras.optimizers import SGD

import json
import argparse
import time

import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
import pickle
import matplotlib.pyplot as plt

# from utils import WeightReader, draw_boxes, decode_netout
# from preprocessing import parse_annotation, BatchGenerator

In [2]:
LABELS = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "+", "-", "times", "div", "(", ")", "[", "]", 
          "{", "}", "!", ","]

IMAGE_H, IMAGE_W = 416, 416
GRID_H,  GRID_W  = 13 , 13
BOX              = 5
CLASS            = len(LABELS)
CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')
OBJ_THRESHOLD    = 0.3 #0.5
NMS_THRESHOLD    = 0.3 #0.45
ANCHORS          = [0.48,0.97, 0.65,1.21, 0.89,1.01, 1.14,0.39, 1.20,1.31]
NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0

TRAIN_DATASET_PATH = os.path.expanduser("~/Workspaces/DeepCalculatorBot/ObjectDetection/dataset/train/")
VALID_DATASET_PATH = os.path.expanduser("~/Workspaces/DeepCalculatorBot/ObjectDetection/dataset/valid/")

EPOCHS           = 100
BATCH_SIZE       = 32
WARM_UP_BATCHES  = 3
TRUE_BOX_BUFFER  = 100

# Create Model

In [3]:
def parse_json(fname):
    """Parse the input profile
    @param fname: input profile path
    @return data: a dictionary with user-defined data for training
    """
    with open(fname) as data_file:
        data = json.load(data_file)
    return data


def write_json(data, fname='./output.json'):
    """Write data to json
    @param data: object to be written
    Keyword arguments:
    fname  -- output filename (default './output.json')
    """
    with open(fname, 'w') as fp:
        json.dump(data, fp, cls=NumpyAwareJSONEncoder)


def print_time(t0, s):
    """Print how much time has been spent
    @param t0: previous timestamp
    @param s: description of this step
    """

    print("%.5f seconds to %s" % ((time.time() - t0), s))
    return time.time()

In [4]:
# define some auxiliary variables and the fire module
sq1x1  = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu   = "relu_"


def fire_module(x, fire_id, squeeze=16, expand=64):

    s_id = 'fire' + str(fire_id) + '/'
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1, kernel_initializer='glorot_uniform')(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)
    
    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1, kernel_initializer='glorot_uniform')(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)
    
    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3, kernel_initializer='glorot_uniform')(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)
    
    x = concatenate([left, right], axis=3, name=s_id + 'concat')
    return x


def SqueezeNet_v2(nb_classes, inputs=(416, 416, 3)):
    
    input_img = Input(shape=inputs)
    
    x = Convolution2D(96, (7, 7), activation='relu', kernel_initializer='glorot_uniform', strides=(2, 2), padding='same', name='conv1')(input_img)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool1')(x)
    
    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size= (3, 3), strides=(2, 2), name='pool3')(x)
    
    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size= (3, 3), strides=(2, 2), name='pool5')(x)
    
    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='fire9/dropout')(x)
    x = Convolution2D(nb_classes, (1, 1), activation='relu', kernel_initializer='glorot_uniform', padding='valid', name='conv10')(x)

    global_avgpool10 = GlobalAveragePooling2D()(x)
    softmax = Activation("softmax", name='softmax')(global_avgpool10)

    return Model(inputs=input_img, outputs=softmax)
  
    
def SqueezeNet(nb_classes, inputs=(224, 224, 3)):
    """ Keras Implementation of SqueezeNet(arXiv 1602.07360)
    @param nb_classes: total number of final categories
    Arguments:
    inputs -- shape of the input images (channel, cols, rows)
    """

    input_img = Input(shape=inputs)
    conv1 = Convolution2D(
        96, (7, 7), activation='relu', kernel_initializer='glorot_uniform',
        strides=(2, 2), padding='same', name='conv1',
        data_format="channels_first")(input_img)
    maxpool1 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool1',
        data_format="channels_first")(conv1)
    fire2_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_squeeze',
        data_format="channels_first")(maxpool1)
    fire2_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand1',
        data_format="channels_first")(fire2_squeeze)
    fire2_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire2_expand2',
        data_format="channels_first")(fire2_squeeze)
    merge2 = Concatenate(axis=1)([fire2_expand1, fire2_expand2])

    fire3_squeeze = Convolution2D(
        16, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_squeeze',
        data_format="channels_first")(merge2)
    fire3_expand1 = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand1',
        data_format="channels_first")(fire3_squeeze)
    fire3_expand2 = Convolution2D(
        64, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire3_expand2',
        data_format="channels_first")(fire3_squeeze)
    merge3 = Concatenate(axis=1)([fire3_expand1, fire3_expand2])

    fire4_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_squeeze',
        data_format="channels_first")(merge3)
    fire4_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand1',
        data_format="channels_first")(fire4_squeeze)
    fire4_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire4_expand2',
        data_format="channels_first")(fire4_squeeze)
    merge4 = Concatenate(axis=1)([fire4_expand1, fire4_expand2])
    maxpool4 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool4',
        data_format="channels_first")(merge4)

    fire5_squeeze = Convolution2D(
        32, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_squeeze',
        data_format="channels_first")(maxpool4)
    fire5_expand1 = Convolution2D(
        128, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand1',
        data_format="channels_first")(fire5_squeeze)
    fire5_expand2 = Convolution2D(
        128, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire5_expand2',
        data_format="channels_first")(fire5_squeeze)
    merge5 = Concatenate(axis=1)([fire5_expand1, fire5_expand2])

    fire6_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_squeeze',
        data_format="channels_first")(merge5)
    fire6_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand1',
        data_format="channels_first")(fire6_squeeze)
    fire6_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire6_expand2',
        data_format="channels_first")(fire6_squeeze)
    merge6 = Concatenate(axis=1)([fire6_expand1, fire6_expand2])

    fire7_squeeze = Convolution2D(
        48, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_squeeze',
        data_format="channels_first")(merge6)
    fire7_expand1 = Convolution2D(
        192, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand1',
        data_format="channels_first")(fire7_squeeze)
    fire7_expand2 = Convolution2D(
        192, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire7_expand2',
        data_format="channels_first")(fire7_squeeze)
    merge7 = Concatenate(axis=1)([fire7_expand1, fire7_expand2])

    fire8_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_squeeze',
        data_format="channels_first")(merge7)
    fire8_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand1',
        data_format="channels_first")(fire8_squeeze)
    fire8_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire8_expand2',
        data_format="channels_first")(fire8_squeeze)
    merge8 = Concatenate(axis=1)([fire8_expand1, fire8_expand2])

    maxpool8 = MaxPooling2D(
        pool_size=(3, 3), strides=(2, 2), name='maxpool8',
        data_format="channels_first")(merge8)
    fire9_squeeze = Convolution2D(
        64, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_squeeze',
        data_format="channels_first")(maxpool8)
    fire9_expand1 = Convolution2D(
        256, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand1',
        data_format="channels_first")(fire9_squeeze)
    fire9_expand2 = Convolution2D(
        256, (3, 3), activation='relu', kernel_initializer='glorot_uniform',
        padding='same', name='fire9_expand2',
        data_format="channels_first")(fire9_squeeze)
    merge9 = Concatenate(axis=1)([fire9_expand1, fire9_expand2])

    fire9_dropout = Dropout(0.5, name='fire9_dropout')(merge9)
    conv10 = Convolution2D(
        nb_classes, (1, 1), activation='relu', kernel_initializer='glorot_uniform',
        padding='valid', name='conv10',
        data_format="channels_first")(fire9_dropout)

    global_avgpool10 = GlobalAveragePooling2D(data_format='channels_first')(conv10)
    softmax = Activation("softmax", name='softmax')(global_avgpool10)

    return Model(inputs=input_img, outputs=softmax)

In [9]:
model = SqueezeNet(nb_classes=len(LABELS), inputs=(1, 416, 416))
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1, 416, 416)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 96, 208, 208) 4800        input_2[0][0]                    
__________________________________________________________________________________________________
maxpool1 (MaxPooling2D)         (None, 96, 103, 103) 0           conv1[0][0]                      
__________________________________________________________________________________________________
fire2_squeeze (Conv2D)          (None, 16, 103, 103) 1552        maxpool1[0][0]                   
__________________________________________________________________________________________________
fire2_expa

In [10]:
model = SqueezeNet_v2(nb_classes=len(LABELS), inputs=(416, 416, 1))
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 416, 416, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 208, 208, 96) 4800        input_3[0][0]                    
__________________________________________________________________________________________________
maxpool1 (MaxPooling2D)         (None, 103, 103, 96) 0           conv1[0][0]                      
__________________________________________________________________________________________________
fire2/squeeze1x1 (Conv2D)       (None, 103, 103, 16) 1552        maxpool1[0][0]                   
__________________________________________________________________________________________________
fire2/relu

# Training

In [24]:
def train_data_generator(data_dir, img_width, img_height, class_mode='categorical', batch_size=32, train_mode=True):
   
    if train_mode:
        datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
    else:
        datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    
    generator = datagen.flow_from_directory(
        data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=class_mode,
        color_mode='grayscale'
    )
    
    return generator

def val_data_generator(val_data_dir, img_width, img_height,
                           class_mode="categorical", batch_size=32):
        """return ImageDataGenerator for validation data"""

        return train_data_generator(
            val_data_dir, img_width, img_height, train_mode=False,
            class_mode=class_mode, batch_size=batch_size)

In [26]:
channels = 3
sgd = SGD(lr=5e-4, decay=0.0002, momentum=0.9)

train_generator = train_data_generator(TRAIN_DATASET_PATH, IMAGE_H, IMAGE_W, batch_size=BATCH_SIZE)
val_generator = val_data_generator(VALID_DATASET_PATH, IMAGE_H, IMAGE_W, batch_size=BATCH_SIZE)

nb_train_samples = train_generator.samples
nb_val_samples = val_generator.samples

print(train_generator.samples)
print(val_generator.samples)

Found 129905 images belonging to 23 classes.
Found 62145 images belonging to 23 classes.
129905
62145


In [27]:
t0 = time.time()
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=EPOCHS,
    validation_data=val_generator,
    nb_val_samples=nb_val_samples
)
print("Model trained in", time.time() - t0, "seconds")
model.save_weights('./squeeze_weights.h5', overwrite=True)

Epoch 1/100


/home/jacopo.gasparetto/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/jacopo.gasparetto/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=4059, epochs=100, validation_steps=62145)`
  import sys


ValueError: Error when checking target: expected softmax to have shape (22,) but got array with shape (23,)

In [6]:
def custom_loss(y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]
    
    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    cell_y = tf.transpose(cell_x, (0,2,1,3,4))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, BOX, 1])
    
    coord_mask = tf.zeros(mask_shape)
    conf_mask  = tf.zeros(mask_shape)
    class_mask = tf.zeros(mask_shape)
    
    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)
    
    """
    Adjust prediction
    """
    ### adjust x and y     
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
    
    ### adjust w and h
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(ANCHORS, [1,1,1,BOX,2])
    
    ### adjust confidence
    pred_box_conf = tf.sigmoid(y_pred[..., 4])
    
    ### adjust class probabilities
    pred_box_class = y_pred[..., 5:]
    
    """
    Adjust ground truth
    """
    ### adjust x and y
    true_box_xy = y_true[..., 0:2] # relative position to the containing cell
    
    ### adjust w and h
    true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically
    
    ### adjust confidence
    true_wh_half = true_box_wh / 2.
    true_mins    = true_box_xy - true_wh_half
    true_maxes   = true_box_xy + true_wh_half
    
    pred_wh_half = pred_box_wh / 2.
    pred_mins    = pred_box_xy - pred_wh_half
    pred_maxes   = pred_box_xy + pred_wh_half       
    
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
    pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)
    
    true_box_conf = iou_scores * y_true[..., 4]
    
    ### adjust class probabilities
    true_box_class = tf.argmax(y_true[..., 5:], -1)
    
    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * COORD_SCALE
    
    ### confidence mask: penelize predictors + penalize boxes with low IOU
    # penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
    true_xy = true_boxes[..., 0:2]
    true_wh = true_boxes[..., 2:4]
    
    true_wh_half = true_wh / 2.
    true_mins    = true_xy - true_wh_half
    true_maxes   = true_xy + true_wh_half
    
    pred_xy = tf.expand_dims(pred_box_xy, 4)
    pred_wh = tf.expand_dims(pred_box_wh, 4)
    
    pred_wh_half = pred_wh / 2.
    pred_mins    = pred_xy - pred_wh_half
    pred_maxes   = pred_xy + pred_wh_half    
    
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_wh[..., 0] * true_wh[..., 1]
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)

    best_ious = tf.reduce_max(iou_scores, axis=4)
    conf_mask = conf_mask + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4]) * NO_OBJECT_SCALE
    
    # penalize the confidence of the boxes, which are reponsible for corresponding ground truth box
    conf_mask = conf_mask + y_true[..., 4] * OBJECT_SCALE
    
    ### class mask: simply the position of the ground truth boxes (the predictors)
    class_mask = y_true[..., 4] * tf.gather(CLASS_WEIGHTS, true_box_class) * CLASS_SCALE       
    
    """
    Warm-up training
    """
    no_boxes_mask = tf.to_float(coord_mask < COORD_SCALE/2.)
    seen = tf.assign_add(seen, 1.)
    
    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, WARM_UP_BATCHES), 
                          lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, 
                                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(ANCHORS, [1,1,1,BOX,2]) * no_boxes_mask, 
                                   tf.ones_like(coord_mask)],
                          lambda: [true_box_xy, 
                                   true_box_wh,
                                   coord_mask])
    
    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))
    nb_class_box = tf.reduce_sum(tf.to_float(class_mask > 0.0))
    
    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(true_box_wh-pred_box_wh)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask)  / (nb_conf_box  + 1e-6) / 2.
    loss_class = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_box_class, logits=pred_box_class)
    loss_class = tf.reduce_sum(loss_class * class_mask) / (nb_class_box + 1e-6)
    
    loss = loss_xy + loss_wh + loss_conf + loss_class
    
    nb_true_box = tf.reduce_sum(y_true[..., 4])
    nb_pred_box = tf.reduce_sum(tf.to_float(true_box_conf > 0.5) * tf.to_float(pred_box_conf > 0.3))

    """
    Debugging code
    """    
    current_recall = nb_pred_box/(nb_true_box + 1e-6)
    total_recall = tf.assign_add(total_recall, current_recall) 

    loss = tf.Print(loss, [tf.zeros((1))], message='Dummy Line \t', summarize=1000)
    loss = tf.Print(loss, [loss_xy], message='Loss XY \t', summarize=1000)
    loss = tf.Print(loss, [loss_wh], message='Loss WH \t', summarize=1000)
    loss = tf.Print(loss, [loss_conf], message='Loss Conf \t', summarize=1000)
    loss = tf.Print(loss, [loss_class], message='Loss Class \t', summarize=1000)
    loss = tf.Print(loss, [loss], message='Total Loss \t', summarize=1000)
    loss = tf.Print(loss, [current_recall], message='Current Recall \t', summarize=1000)
    loss = tf.Print(loss, [total_recall/seen], message='Average Recall \t', summarize=1000)
    
    return loss

In [7]:
def normalize(image):
    return image / 255.

In [8]:
generator_config = {
    'IMAGE_H'         : IMAGE_H, 
    'IMAGE_W'         : IMAGE_W,
    'GRID_H'          : GRID_H,  
    'GRID_W'          : GRID_W,
    'BOX'             : BOX,
    'LABELS'          : LABELS,
    'CLASS'           : len(LABELS),
    'ANCHORS'         : ANCHORS,
    'BATCH_SIZE'      : BATCH_SIZE,
    'TRUE_BOX_BUFFER' : TRUE_BOX_BUFFER,
}

In [9]:
train_img_path = "/home/jacopo.gasparetto/Workspaces/DeepCalculatorBot/ObjectDetection/xml_dataset/features/"
train_labels_path = "/home/jacopo.gasparetto/Workspaces/DeepCalculatorBot/ObjectDetection/xml_dataset/labels/"
valid_img_path = "/home/jacopo.gasparetto/Workspaces/DeepCalculatorBot/ObjectDetection/xml_dataset/features/"
valid_labels_path = "/home/jacopo.gasparetto/Workspaces/DeepCalculatorBot/ObjectDetection/xml_dataset/labels/"

train_imgs, seen_train_labels = parse_annotation(train_labels_path, train_img_path, labels=LABELS)
train_batch = BatchGenerator(train_imgs, generator_config, norm=normalize)

valid_imgs, seen_valid_labels = parse_annotation(valid_labels_path, valid_img_path, labels=LABELS)
valid_batch = BatchGenerator(valid_imgs, generator_config, norm=normalize)

In [10]:
train_batch.load_annotation(0)

array([[ 83, 275, 123, 293,  11],
       [ 86, 230, 104, 268,   6],
       [115,  64, 147,  96,   2],
       [244, 324, 263, 370,  18],
       [ 80, 176, 108, 212,   3]])

**Setup a few callbacks and start the training**

In [11]:
early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=0.001, 
                           patience=3, 
                           mode='min', 
                           verbose=1)

checkpoint = ModelCheckpoint('weights_coco.h5', 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min', 
                             period=1)

In [12]:
log_dir = os.path.expanduser('~/Workspaces/DeepCalculatorBot/ObjectDetection/logs/')
tb_counter  = len([log for log in os.listdir(log_dir) if 'coco_' in log]) + 1
tensorboard = TensorBoard(log_dir=os.path.expanduser(log_dir) + 'coco_' + '_' + str(tb_counter), 
                          histogram_freq=0, 
                          write_graph=True, 
                          write_images=True)

optimizer = Adam(lr=0.5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optimizer = SGD(lr=1e-4, decay=0.0005, momentum=0.9)
#optimizer = RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss=custom_loss, optimizer=optimizer)

model.fit_generator(generator        = train_batch, 
                    steps_per_epoch  = len(train_batch), 
                    epochs           = 100, 
                    verbose          = 1,
                    validation_data  = valid_batch,
                    validation_steps = len(valid_batch),
                    callbacks        = [early_stop, checkpoint, tensorboard], 
                    max_queue_size   = 3)

Epoch 1/100
313/313 [==============================] - 870s 3s/step - loss: 10.3148 - val_loss: 3.6099

Epoch 00001: val_loss improved from inf to 3.60992, saving model to weights_coco.h5
Epoch 2/100
313/313 [==============================] - 850s 3s/step - loss: 3.3019 - val_loss: 3.2523

Epoch 00002: val_loss improved from 3.60992 to 3.25227, saving model to weights_coco.h5
Epoch 3/100
313/313 [==============================] - 849s 3s/step - loss: 3.9337 - val_loss: 3.1458

Epoch 00003: val_loss improved from 3.25227 to 3.14576, saving model to weights_coco.h5
Epoch 4/100
313/313 [==============================] - 852s 3s/step - loss: 3.0432 - val_loss: 2.9946

Epoch 00004: val_loss improved from 3.14576 to 2.99463, saving model to weights_coco.h5
Epoch 5/100
313/313 [==============================] - 851s 3s/step - loss: 2.9614 - val_loss: 2.9306

Epoch 00005: val_loss improved from 2.99463 to 2.93059, saving model to weights_coco.h5
Epoch 6/100
313/313 [===========================

# Perform detection on image

In [13]:
model.load_weights("weights_coco.h5")

In [28]:
# image = cv2.imread('/home/jacopo.gasparetto/Workspaces/DeepCalculatorBot/ObjectDetection/fake_dataset/features/52.png')
image = cv2.imread('/home/jacopo.gasparetto/Workspaces/DeepCalculatorBot/ObjectDetection/test_img.jpg', cv2.IMREAD_GRAYSCALE)
dummy_array = np.zeros((1,1,1,1,TRUE_BOX_BUFFER,4))

plt.figure(figsize=(10,10))
print(image.shape)
input_image = cv2.resize(image, (416, 416))
input_image = input_image / 255.
input_image = input_image[:,:,::-1]
input_image = np.expand_dims(input_image, 0)

netout = model.predict([input_image, dummy_array])
boxes = decode_netout(netout[0], 
                      obj_threshold=OBJ_THRESHOLD,
                      nms_threshold=NMS_THRESHOLD,
                      anchors=ANCHORS, 
                      nb_class=CLASS)
image = draw_boxes(image, boxes, labels=LABELS)

plt.imshow(image[:,:,::-1]); plt.show()

(1280, 960)


IndexError: too many indices for array

<Figure size 720x720 with 0 Axes>